In [7]:
%reload_ext autoreload
%autoreload 2

import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
pd.options.display.max_columns = 999
pd.options.display.max_rows = 1999
pd.options.display.max_colwidth = 100

import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot, offline
import plotly.express as px
import seaborn as sns
import matplotlib.pyplot as plt

import requests

from os import listdir
import unicodedata

from paths import *
from scripts.manipulation import remove_acentos
from scripts.manipulation import normalize_cols
from scripts import scrap_data
from scripts import manipulation
from scripts import io
from scripts import vis_graphs

from datetime import datetime
today = datetime.today().strftime('%Y-%m-%d')

import yaml


## Casos

In [8]:
sp_full = pd.read_csv('https://raw.githubusercontent.com/seade-R/dados-covid-sp/master/data/dados_covid_sp.csv', sep=';')
sp_full['nome_drs'] = np.where(sp_full['nome_munic']=='São Paulo','Município de São Paulo',sp_full['nome_drs'])

cols = ['datahora','nome_munic','casos','obitos','nome_drs']
sp = sp_full[cols][sp_full['nome_drs'].notnull()]

sp = sp.groupby(by=['nome_drs','datahora'], as_index=False).sum().sort_values(by=['nome_drs','datahora'])

estado = sp.groupby(by=['datahora'], as_index=False).sum().sort_values(by=['datahora'])
estado['nome_drs'] = 'Estado de São Paulo'
estado = estado[sp.columns]

sp = pd.concat([sp,estado],axis=0)

In [9]:
sp['casos'] = sp['casos'].astype(int)
sp['obitos'] = sp['obitos'].astype(int)

In [10]:
sp['nome_drs'].unique()

array(['Araraquara', 'Araçatuba', 'Baixada Santista', 'Barretos', 'Bauru',
       'Campinas', 'Franca', 'Grande São Paulo', 'Marília',
       'Município de São Paulo', 'Piracicaba', 'Presidente Prudente',
       'Registro', 'Ribeirão Preto', 'Sorocaba', 'São José do Rio Preto',
       'São João da Boa Vista', 'Taubaté', 'Estado de São Paulo'],
      dtype=object)

In [11]:
dd_final_all = pd.DataFrame()

for drs in sp['nome_drs'].unique():

    tbt = sp[sp['nome_drs']==drs]

    tbt['casos_shift'] = tbt['casos'].shift(7)
    tbt['obitos_shift'] = tbt['obitos'].shift(7)

    tbt['casos_7'] = tbt['casos'] - tbt['casos_shift']
    tbt['obitos_7'] = tbt['obitos'] - tbt['obitos_shift']

    tbt['casos_7_shift'] = tbt['casos_7'].shift(7)
    tbt['obitos_7_shift'] = tbt['obitos_7'].shift(7)

    tbt['casos_var'] = tbt['casos_7']/tbt['casos_7_shift']
    tbt['obitos_var'] = tbt['obitos_7']/tbt['obitos_7_shift']

    dd = tbt[['datahora','nome_drs','casos_var','obitos_var']]

    dd_final_all = pd.concat([dd_final_all,dd], axis=0)

    
mask = dd_final_all['datahora'] == max(dd_final_all['datahora'])
dd_final = dd_final_all[mask]

## Internacoes

In [12]:
sp_cv = pd.read_csv('https://github.com/seade-R/dados-covid-sp/raw/master/data/plano_sp_leitos_internacoes.csv', sep=';')
sp_cv['internacoes_var'] = sp_cv['internacoes_7d']/sp_cv['internacoes_7d_l']
cols = ['datahora','nome_drs','leitos_pc','internacoes_var']
uti_all = sp_cv[cols]

last_data =  max(uti_all['datahora'])


mask = uti_all['datahora'] == last_data
df_final = uti_all[mask][cols]
df_final['nome_drs'] = df_final['nome_drs'].apply(lambda x: x[7:] if x[:3]=='DRS' else x).str.strip()


## Ocupacao UTI

In [13]:
uti = pd.read_csv('../data/sp_gov/uti_ocupacao.csv', sep=';')
uti_final = uti[uti['datahora']==max(uti['datahora'])]


uti_final['datahora'] = last_data

## Add Fases

In [14]:

df = pd.merge(dd_final,df_final,on=['datahora','nome_drs'],how='outer').merge(uti_final,on=['datahora','nome_drs'],how='outer')
mask = df['nome_drs'].str.startswith('Grande')
df = df[np.logical_not(mask)]
df['leitos_pc'] = df['leitos_pc'].str.replace(',','.')

df = df.round(2)

def uti_fase(x):
    if x>=80:
        fase=1
    elif (x <80) & (x>=70):
        fase=2
    elif (x <70) & (x>=60):
        fase=3
    else:
        fase = 4
    return fase

def leitos_fase(x):
    if x<=3:
        fase=1
    elif (x >3) & (x<=5):
        fase=2
    else:
        fase = 4
    return fase


O = df['uti_var'].apply(lambda x: uti_fase(x))

L = df['leitos_pc'].str.replace(',','.').astype(float).apply(lambda x: leitos_fase(x))


c1 = (O*4 + L)/5

df['Capacidade Hospitalar'] = c1.astype(int)

def casos_fase(x):
    if x>=2:
        fase=1
    elif (x <2) & (x>=1):
        fase=3
    else:
        fase = 4
    return fase

def internacoes_fase(x):
    if x>=1.5:
        fase=1
    elif (x <1.5) & (x>=1):
        fase=2
    elif (x <1) & (x>=0.5):
        fase=3
    else:
        fase = 4
    return fase

def obitos_fase(x):
    if x>=2:
        fase=1
    elif (x <2) & (x>=1):
        fase=2
    elif (x <1) & (x>=0.5):
        fase=3
    else:
        fase = 4
    return fase
NC = df['casos_var'].apply(lambda x: casos_fase(x))

NI = df['internacoes_var'].apply(lambda x: internacoes_fase(x))

NO = df['obitos_var'].apply(lambda x: obitos_fase(x))

c2 = (NC + 3*NI + NO)/5

df['Evolução da Pandemia'] = c2

df['casos_var'] = df['casos_var'].round(2).astype(str)
df['obitos_var'] = df['obitos_var'].round(2).astype(str)
df['internacoes_var'] = df['internacoes_var'].round(2).astype(str)
df['uti_var'] = df['uti_var'].round(2).astype(str)

df['Classif. Final'] = df[['Capacidade Hospitalar','Evolução da Pandemia']].min(axis=1)

df['Classif. Final'] = df['Classif. Final'].astype(int).astype(str)
df['Evolução da Pandemia'] = df['Evolução da Pandemia'].astype(int).astype(str)
df['Capacidade Hospitalar'] = df['Capacidade Hospitalar'].astype(int).astype(str)

## Padronize 

In [15]:

cols = ['datahora','nome_drs','Capacidade Hospitalar','uti_var','leitos_pc','Evolução da Pandemia','casos_var','internacoes_var','obitos_var','Classif. Final']


rename_drs = {
    'Araraquara':'DRS 03 - Araraquara',
    'Araçatuba':'DRS 02 - Araçatuba',
    'Baixada Santista':'DRS 04 - Baixada Santista',
    'Barretos':'DRS 05 - Barretos',
    'Bauru':'DRS 06 - Bauru',
    'Campinas':'DRS 07 - Campinas',
    'Estado de São Paulo':'Estado de São Paulo',
    'Franca':'DRS 08 - Franca',
    'Marília':'DRS 09 - Marília',
    'Município de São Paulo':'DRS 01 - Município de São Paulo',
    'Piracicaba':'DRS 10 - Piracicaba',
    'Presidente Prudente':'DRS 11 - Presidente Prudente',
    'Registro':'DRS 12 - Registro',
    'Ribeirão Preto':'DRS 13 - Ribeirão Preto',
    'Sorocaba':'DRS 16 - Sorocaba',
    'São José do Rio Preto':'DRS 15 - São José do Rio Preto',
    'São João da Boa Vista':'DRS 14 - São João da Boa Vista',
    'Taubaté':'DRS 17 - Taubaté'
}

df['nome_drs'] = df['nome_drs'].map(rename_drs)

cols_rename ={
    'datahora':'Data',
    'nome_drs':'DRS',
    'casos_var':'Variação de casos',
    'obitos_var':'Variação de óbitos',
    'leitos_pc':'Leitos COVID/100 mil hab',
    'internacoes_var':'Variação internações',
    'uti_var':'Ocupação leitos UTI COVID',
}




df = df[cols].rename(columns=cols_rename).sort_values(by='DRS')

df = pd.concat([df.tail(1),df.head(17)],axis=0).reset_index(drop=True)


## Colored Table

In [16]:
def cases_color(val):
    if val == '1':
        color = 'red'
    elif val=='2':
        color = 'orange'
    elif val=='3':
        color = 'yellow'
    else:
        color = 'green'
    return 'background-color: %s' % color

df.style.applymap(cases_color, subset=pd.IndexSlice[:, ['Capacidade Hospitalar','Evolução da Pandemia','Classif. Final']] )

,Data,DRS,Capacidade Hospitalar,Ocupação leitos UTI COVID,Leitos COVID/100 mil hab,Evolução da Pandemia,Variação de casos,Variação internações,Variação de óbitos,Classif. Final
0,2020-07-13,Estado de São Paulo,3,65.0,20.33,2,1.08,1.04,1.02,2
1,2020-07-13,DRS 01 - Município de São Paulo,3,66.6,31.09,3,0.91,0.99,0.98,3
2,2020-07-13,DRS 02 - Araçatuba,3,60.4,13.07,2,1.36,1.28,1.0,2
3,2020-07-13,DRS 03 - Araraquara,4,31.2,9.74,2,1.3,1.23,0.89,2
4,2020-07-13,DRS 04 - Baixada Santista,4,49.4,26.29,3,0.87,0.96,0.81,3
5,2020-07-13,DRS 05 - Barretos,2,78.0,16.47,2,1.2,0.8,1.17,2
6,2020-07-13,DRS 06 - Bauru,3,65.6,10.59,2,1.11,1.21,1.26,2
7,2020-07-13,DRS 07 - Campinas,1,80.4,18.61,2,1.2,1.12,1.22,1
8,2020-07-13,DRS 08 - Franca,1,84.7,7.63,2,1.83,0.97,2.4,1
9,2020-07-13,DRS 09 - Marília,4,10.9,10.97,2,1.46,1.18,1.0,2


In [23]:
mask = sp_cv['nome_drs']=='DRS 17 Taubaté'
sp_cv[mask].tail(30)

,datahora,nome_drs,total_covid_uti_mm7d,pop,leitos_pc,internacoes_7d,internacoes_7d_l,internacoes_7v7,internacoes_var
619,2020-06-14,DRS 17 Taubaté,"301,00",2489629,"12,09",418,399,"4,76",1.047619
642,2020-06-15,DRS 17 Taubaté,"300,43",2489629,"12,07",431,414,"4,11",1.041063
665,2020-06-16,DRS 17 Taubaté,"301,00",2489629,"12,09",426,416,"2,40",1.024038
688,2020-06-17,DRS 17 Taubaté,"301,29",2489629,"12,10",416,428,"-2,80",0.971963
711,2020-06-18,DRS 17 Taubaté,"304,00",2489629,"12,21",403,434,"-7,14",0.928571
734,2020-06-19,DRS 17 Taubaté,"308,14",2489629,"12,38",408,435,"-6,21",0.937931
757,2020-06-20,DRS 17 Taubaté,"312,57",2489629,"12,55",427,421,"1,43",1.014252
780,2020-06-21,DRS 17 Taubaté,"319,57",2489629,"12,84",425,418,"1,67",1.016746
803,2020-06-22,DRS 17 Taubaté,"327,57",2489629,"13,16",450,431,"4,41",1.044084
826,2020-06-23,DRS 17 Taubaté,"335,86",2489629,"13,49",500,426,"17,37",1.173709


In [25]:
mask = dd_final_all['nome_drs']=='Taubaté'
dd_final_all[mask].tail(30)

,datahora,nome_drs,casos_var,obitos_var
2490,2020-06-14,Taubaté,1.083700,1.038462
2491,2020-06-15,Taubaté,1.014124,1.160000
2492,2020-06-16,Taubaté,1.168303,0.766667
2493,2020-06-17,Taubaté,0.985372,0.843750
2494,2020-06-18,Taubaté,0.696933,1.000000
2495,2020-06-19,Taubaté,1.268804,0.965517
2496,2020-06-20,Taubaté,1.436340,1.461538
2497,2020-06-21,Taubaté,1.517615,1.407407
2498,2020-06-22,Taubaté,1.632312,1.275862
2499,2020-06-23,Taubaté,1.408163,1.826087
